In [90]:
import os
import sys

PROJECT_ROOT = os.path.join(os.path.abspath(os.path.dirname('./../../')), 'src')

sys.path.append(PROJECT_ROOT)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
conf = {'groups_desc': 'data/small_group_description.csv',
    'train_x': 'data/transactions_train.csv',
    'test_x': 'data/transactions_test.csv',
    'train_target': 'data/train_target.csv',
    'test_target': 'data/test.csv'}

In [92]:
from data_utils import *

In [277]:
%%time

ds = DataSource(conf)
ds.read_data()
ds.validation_split()

Reading groups_desc...
Reading train_x...
Reading test_x...
Reading train_target...
Reading test_target...
Validation split: by clientID
Wall time: 1min 29s


In [94]:
train_target = ds.get_data('train_target')
train_target.set_index('client_id', inplace=True)

In [95]:
ds.get_data('groups_desc')

,small_group,small_group_code
0,Зоотовары,0
1,Сетевые супермаркеты и продуктовые магазины,1
2,Обслуживание существующего автомобиля (АЗС),2
3,Оплата телефона и связи,3
4,Аптеки,4
5,Авиабилеты,5
6,Duty-free,6
7,Страховка,7
8,Спорт,8
9,Универсальные товары,9


In [96]:
train_x = ds.get_data('train_x')
print(train_x.head())


   client_id  trans_date  small_group  amount_rur
0       3694          26            1      10.006
1       3694          29            3      54.955
2       3694          31            1      10.945
3       3694          34           36      61.721
4       3694          36           25       4.579


In [97]:
train_x['month_number'] = train_x['trans_date']//30

In [98]:
train_x.head()

,client_id,trans_date,small_group,amount_rur,month_number
0,3694,26,1,10.006,0
1,3694,29,3,54.955,0
2,3694,31,1,10.945,1
3,3694,34,36,61.721,1
4,3694,36,25,4.579,1


In [99]:
grouped = train_x.groupby(['client_id', 'month_number'], as_index=False)

In [100]:

client_per_month_spendings = train_x.groupby(['client_id', 'month_number']).agg({"amount_rur": "sum"}).groupby('client_id').agg({'amount_rur':'mean'})

In [101]:
print(train_x.groupby(['client_id']).size())


client_id
4         720
6         730
10        706
11        812
13        754
14       1097
17       1053
19        721
20        821
25        996
27        811
28        765
31        834
32       1035
33        908
34       1033
36        819
37        772
38        720
39        760
40        916
42        792
43        917
44        714
46        866
47        749
48        920
49        868
50        970
51        791
         ... 
49949     732
49950    1001
49951     915
49955     856
49957     870
49959     963
49961    1083
49962     829
49963     747
49967     702
49968     775
49971     704
49972    1082
49973     904
49974    1129
49975    1030
49977     721
49978     960
49980    1023
49981     757
49982     957
49983     904
49987    1120
49988     777
49989     706
49990     721
49995     969
49996    1138
49997     715
49998     782
Length: 24145, dtype: int64


In [102]:
grouped_by_group = train_x.groupby(['client_id', 'small_group'], as_index = False)['small_group'].agg({'freq':'count'})
client_per_max_frequent_category  = grouped_by_group.loc[grouped_by_group.groupby('client_id')['freq'].idxmax()][['client_id','small_group']]
client_per_max_frequent_category.columns = ['client_id', 'freq_group']
client_per_max_frequent_category.set_index('client_id', inplace=True)
print(client_per_max_frequent_category.head())


           freq_group
client_id            
4                   1
6                   1
10                  1
11                  1
13                 11


In [181]:
# Вычислием среднюю максимальную транзакцию по клиентам. Далее найдем среднее отклонение максим транзакций по клиентам. 
# Это позволит найти "большие" транзакции , как mean + std. у нас не гаусово распределение, но все же оперделим значение "большой" транзакции 
# именно так.
grouped_by_max = train_x.groupby(['client_id'], as_index = False).agg({'amount_rur':'max'})
print(grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] )
large_transactions = grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] /5
print(large_transactions)

2765.900111078898
553.1800222157797


In [182]:
len(train_x[train_x['amount_rur'] > large_transactions]['client_id'].unique())

20288

In [189]:
grouped_by_min = train_x.groupby(['client_id'], as_index = False).agg({'amount_rur':'min'})
small_transactions = grouped_by_min.agg({'amount_rur':'mean'})['amount_rur'] + grouped_by_min.agg({'amount_rur':'std'})['amount_rur']
print(small_transactions)

0.8787477259259386


In [190]:
len(train_x[train_x['amount_rur'] < small_transactions]['client_id'].unique())

21109

In [191]:
client_per_small_count = train_x[train_x['amount_rur'] < small_transactions].groupby(['client_id'])['trans_date'].agg({'small_num':'count'})

In [203]:
client_per_large_count = train_x[train_x['amount_rur'] > large_transactions].groupby(['client_id'])['trans_date'].agg({'large_num':'count'})

In [109]:
clean_trans_per_month = train_x.groupby(['client_id', 'month_number'])['amount_rur'].agg({"tran_num_mean": "count"}).groupby('client_id').agg({'tran_num_mean':'mean'})

In [193]:
client_max = train_x.groupby(['client_id'])['amount_rur'].agg({"max": "max"})
print(client_max.head())

                max
client_id          
4          1341.802
6           315.781
10          654.893
11         2105.058
13         1190.322


In [204]:
print(client_per_month_spendings.shape)
print(client_per_max_frequent_category.shape)
print(clean_trans_per_month.shape)
print(client_per_small_count.shape)
print(client_per_large_count.shape)

(24145, 1)
(24145, 1)
(24145, 1)
(21109, 1)
(20288, 1)


In [205]:
res_df = client_per_month_spendings.join(client_max).join(clean_trans_per_month).join(client_per_small_count).join(client_per_large_count)
res_df.fillna(0, inplace=True)
print(res_df.head())

           amount_rur       max  tran_num_mean  small_num  large_num
client_id                                                           
4          1136.16484  1341.802          28.80        3.0        2.0
6           628.82956   315.781          29.20        1.0        0.0
10         1376.77460   654.893          28.24        1.0        2.0
11         1071.57616  2105.058          32.48       28.0        7.0
13          693.49868  1190.322          30.16        8.0        1.0


In [206]:
from sklearn import preprocessing

column_names_to_normalize = ["amount_rur", "max", "tran_num_mean", "small_num", "large_num"]
x = res_df[column_names_to_normalize].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = res_df.index)
res_df[column_names_to_normalize] = df_temp

print(res_df.head(30))

           amount_rur       max  tran_num_mean  small_num  large_num
client_id                                                           
4            0.004118  0.003215       0.036727   0.013043   0.003053
6            0.001931  0.000650       0.055090   0.004348   0.000000
10           0.005155  0.001498       0.011018   0.004348   0.003053
11           0.003840  0.005123       0.205669   0.121739   0.010687
13           0.002210  0.002836       0.099162   0.034783   0.001527
14           0.004002  0.002779       0.812957   0.021739   0.004580
17           0.009279  0.014092       0.648224   0.008696   0.010687
19           0.007315  0.004984       0.038563   0.017391   0.009160
20           0.013014  0.007895       0.222196   0.043478   0.035115
25           0.012539  0.006271       0.543553   0.013043   0.025954
27           0.003371  0.001135       0.203832   0.034783   0.000000
28           0.003478  0.003980       0.119361   0.008696   0.004580
31           0.010891  0.012248   

In [208]:
print(res_df.head(30))

           amount_rur       max  tran_num_mean  small_num  large_num
client_id                                                           
4            0.004118  0.003215       0.036727   0.013043   0.003053
6            0.001931  0.000650       0.055090   0.004348   0.000000
10           0.005155  0.001498       0.011018   0.004348   0.003053
11           0.003840  0.005123       0.205669   0.121739   0.010687
13           0.002210  0.002836       0.099162   0.034783   0.001527
14           0.004002  0.002779       0.812957   0.021739   0.004580
17           0.009279  0.014092       0.648224   0.008696   0.010687
19           0.007315  0.004984       0.038563   0.017391   0.009160
20           0.013014  0.007895       0.222196   0.043478   0.035115
25           0.012539  0.006271       0.543553   0.013043   0.025954
27           0.003371  0.001135       0.203832   0.034783   0.000000
28           0.003478  0.003980       0.119361   0.008696   0.004580
31           0.010891  0.012248   

In [221]:
train_dataset = Ds(data = res_df, labels = train_target)
print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=2000, shuffle=True)  

           bins  amount_rur       max  tran_num_mean  small_num  large_num
client_id                                                                 
1046          0    0.006557  0.002887       0.216687   0.004348   0.013740
34089         2    0.003849  0.001817       0.150579   0.052174   0.003053
34848         1    0.001940  0.000135       0.506826   0.069565   0.000000
47076         3    0.001374  0.001214       0.154251   0.060870   0.000000
10938         2    0.002294  0.001216       0.806148   0.013043   0.000000
42965         1    0.006287  0.000979       0.780439   0.000000   0.000000
21594         0    0.004509  0.001277       0.315848   0.008696   0.001527
7952          1    0.005988  0.002905       0.123034   0.008696   0.003053
33810         0    0.003719  0.002122       0.097325   0.017391   0.003053
11061         2    0.002631  0.005940       0.462754   0.065217   0.001527
30905         0    0.002047  0.001221       0.192814   0.043478   0.000000
29440         3    0.0042

In [244]:
def prepare_featured_data(df):
    df['month_number'] = df['trans_date']//30
    per_month = df.groupby(['client_id', 'month_number']).agg({"amount_rur": "sum"}).groupby('client_id').agg({'amount_rur':'mean'})
    #print(per_month.head())
    large_count = df[df['amount_rur'] > 500].groupby(['client_id'])['trans_date'].agg({'large_num':'count'})
    #print(large_count.head())
    small_count = df[df['amount_rur'] < 1].groupby(['client_id'])['trans_date'].agg({'small_num':'count'})
    #print(small_count.head())
    trans_per_month = df.groupby(['client_id', 'month_number'])['amount_rur'].agg({"tran_num_mean": "count"}).groupby('client_id').agg({'tran_num_mean':'mean'})
    #print(trans_per_month.head())
    client_max = df.groupby(['client_id'])['amount_rur'].agg({"max": "max"})
    #print(client_max.head())
    res = per_month.join(client_max).join(trans_per_month).join(small_count).join(large_count)
    res.fillna(0, inplace=True)
    return res
    

In [245]:
def normalize(column_names_to_normalize, df):
    x = df[column_names_to_normalize].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    
    df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df.index)
    df[column_names_to_normalize] = df_temp
    return df

In [246]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden0 = nn.Linear(5, 8)
        self.hidden1 = nn.Linear(8, 16)
        self.hidden2 = nn.Linear(16, 32)
        self.bn1 = nn.BatchNorm1d(64)
        self.output = nn.Linear(32, 4)

        
    def forward(self, x):
        x = F.relu(self.hidden0(x)) 
        x = F.relu(self.hidden1(x)) 
        x = F.relu(self.hidden2(x)) 
        return F.log_softmax(self.output(x))
        

In [247]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torchvision

class Ds(Dataset):
    
    def __init__(self, data, labels):
        self.data = labels.join(data)
        print(self.data.head(30))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data.iloc[index, 1:]
        y = self.data.iloc[index, 0]

        return torch.tensor(x), torch.tensor(y)

In [286]:
train_y = ds.get_data('train_target')
#print(train_y.head())
#train_y.set_index('client_id', inplace=True)

train_x = ds.get_data('train_x')

valid_y = train_y.iloc[20000:, : ]
print(valid_y.head())
#train_y = train_y.iloc[:20000, : ]
processed = normalize(["amount_rur", "max", "tran_num_mean", "small_num", "large_num"], prepare_featured_data(train_x))
valid_processed = processed.iloc[20000:, : ]
print(valid_processed.head())
#processed = processed.iloc[:20000, : ]

train_dataset = Ds(data = processed, labels = train_y)
train_loader = DataLoader(train_dataset, batch_size=2000, shuffle=True)  

           bins
client_id      
14410         1
35898         3
9192          0
25355         0
42168         0
           amount_rur       max  tran_num_mean  small_num  large_num
client_id                                                           
41504        0.003651  0.001189       0.657405   0.007117   0.001522
41507        0.018967  0.056461       0.433373   0.039146   0.030441
41510        0.001474  0.000378       0.084471   0.035587   0.000000
41512        0.012782  0.006330       0.269940   0.032028   0.036530
41513        0.003564  0.001851       0.255250   0.021352   0.007610
           bins  amount_rur       max  tran_num_mean  small_num  large_num
client_id                                                                 
1046          0    0.006557  0.002887       0.216687   0.003559   0.018265
34089         2    0.003849  0.001817       0.150579   0.056940   0.003044
34848         1    0.001940  0.000135       0.506826   0.064057   0.000000
47076         3    0.001374  0

In [291]:
train_y = train_y.iloc[:20000, : ]
processed = processed.iloc[:20000, :]

In [249]:
import torch 
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda"),
else:
    device = torch.device("cpu")

In [250]:
import numpy as np
import torch.optim as optim

learning_rate = 0.01
momentum = 0.5

network = Net().to(device)
optimizer = optim.Adam(network.parameters(), lr=learning_rate)

for epoch in range(1, 10):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #if batch_idx > 2460:
            #output = network(data.cuda())
            #pred = output.data.max(1, keepdim=True)[1]
            #print('ACURACY: {}'.format((pred.flatten() == target).sum()))
       # else:
            optimizer.zero_grad()
            output = network(data.to(device))
            loss = F.cross_entropy(output, target.to(device)).to(device)
            loss.backward()
            optimizer.step()
    print('Train Epoch: {} \tLoss: {:.6f}'.format(
            epoch, loss.item()))

Train Epoch: 1 	Loss: 1.380920
Train Epoch: 2 	Loss: 1.374123
Train Epoch: 3 	Loss: 1.372923
Train Epoch: 4 	Loss: 1.348212
Train Epoch: 5 	Loss: 1.345523
Train Epoch: 6 	Loss: 1.275716
Train Epoch: 7 	Loss: 1.309435
Train Epoch: 8 	Loss: 1.305176
Train Epoch: 9 	Loss: 1.246177


In [171]:
print(len(res_df[res_df['large_num']>0]))

11972


In [292]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)


clf.fit(processed[["amount_rur", "max", "tran_num_mean", "small_num", "large_num"]].values,train_y['bins'].values)
clf.score(valid_processed[["amount_rur", "max", "tran_num_mean", "small_num", "large_num"]].values,valid_y['bins'].values)

0.2554885404101327

In [283]:
test_y = ds.get_data('test_target')
print(ds.get_data('test_target').iloc[:, 0])
#test_y.set_index('client_id', inplace=True)
#test_x = ds.get_data('test_x')

#test_processed = normalize(["amount_rur", "max", "tran_num_mean", "small_num", "large_num"], prepare_featured_data(test_x))
#clf.score(test_processed[["amount_rur", "max", "tran_num_mean", "small_num", "large_num"]].values,test_y.values)

0        28571
1        27046
2        13240
3        19974
4        10505
5        36581
6        13137
7        29069
8        14334
9         7432
10       44221
11       40799
12       36456
13         818
14       35103
15       48997
16        4975
17        4013
18       30955
19       26778
20         709
21        9410
22       15212
23       11347
24         766
25          55
26        2493
27       46885
28       39401
29        8934
         ...  
19970     5048
19971    21095
19972    29891
19973     6069
19974    39294
19975    41915
19976    44830
19977    44923
19978    25431
19979    27672
19980    21382
19981     6221
19982    30173
19983    28339
19984    24920
19985    17978
19986    16100
19987    41833
19988    26651
19989    37973
19990    14102
19991    26308
19992      910
19993    48964
19994    13582
19995     2565
19996    31255
19997    31539
19998     4288
19999    15323
Name: client_id, Length: 20000, dtype: int64


In [274]:
print(test_y.head())

Empty DataFrame
Columns: []
Index: [28571, 27046, 13240, 19974, 10505]
